In [1]:
from selenium import webdriver
import json
import time
import pandas as pd
pd.options.display.max_columns = None
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
from IPython.display import display, clear_output
import random

In [2]:
driver = webdriver.Chrome()

In [ ]:
models_links = []
next_button = driver.find_element_by_class_name('pager__next')

while True:
    time.sleep(1)
    elem = driver.find_element_by_class_name('catalog-all-text-list')
    models_links+= [x.get_attribute('href') for x in elem.find_elements_by_tag_name('a')]
    time.sleep(1)
    if next_button.is_enabled():
        next_button.click()
    else:
        break
    
    
    
    
with open('models_links.json', 'w') as out_file:
    json.dump({'links': models_links}, out_file)

In [ ]:
with open('models_links.json', 'r') as models_links_file:
    models_links = json.load(models_links_file)['links']

In [ ]:
all_models_links = []
for models_link in models_links:
    driver.get(models_link)
    elem = driver.find_elements_by_class_name('catalog-all-text-list__desc')
    for e in elem:
        models_types = e.find_elements_by_class_name('mosaic__title')
        for model_type in models_types:
            all_models_links.append(model_type.get_attribute('href'))
with open('all_models_links.json', 'w') as out_file:
    json.dump({'links': all_models_links}, out_file)

In [3]:
with open('all_models_links.json', 'r') as all_models_links_file:
    all_models_links = json.load(all_models_links_file)['links']

In [ ]:
data_spec = pd.DataFrame()

for link in all_models_links:
    driver.get(link)
    brand = driver.find_element_by_class_name('search-form-v2-mmm__breadcrumbs-item_type_mark').text
    model = driver.find_element_by_class_name('search-form-v2-mmm__breadcrumbs-item_type_model').text
    generation = driver.find_element_by_class_name('search-form-v2-mmm__breadcrumbs-item_type_generation').text
    config = driver.find_element_by_class_name('search-form-v2-mmm__breadcrumbs-item_type_configuration').text
    

    models_repres = driver.find_element_by_class_name('catalog-table__inner').find_elements_by_class_name('link_theme_auto')
    for repres in models_repres:
        repres.click()
        spec_dict = {}
        spec_dict['Марка'] = brand
        spec_dict['Модель'] = model
        spec_dict['Поколение'] = generation
        spec_dict['Тип кузова'] =config
        #считываем характеристики
        comp = driver.find_elements_by_link_text('Комплектации')
        if len(comp) != 0:
            comp[0].click()
            # cчитываем комплектацию
            driver.find_element_by_link_text('Характеристики').click()
    

In [ ]:
driver.get(all_models_links[2])

In [ ]:
# узнать цену на новые в первом разделе
# марку, модель и тип кузова, поколение
# парсить комплектацию


for group in driver.find_elements_by_class_name('catalog__details-group'):
    a = group.find_elements_by_class_name('list-values__label')
    b = group.find_elements_by_class_name('list-values__value')
    for x, y in zip(a,b):
        spec_dict[x.text] = [y.text]
        
    
    temp_data = pd.DataFrame.from_dict(spec_dict)
    data_spec = data_spec.append(temp_data, sort=False)
    data_spec = data_spec.reset_index()

In [ ]:
#Комплектация
for group in driver.find_elements_by_class_name('catalog__package-group'):
    a = group.find_element_by_class_name('catalog__h3')
    b = group.find_elements_by_class_name('catalog__package-list-i')
    print(a.text)
    for x in b:
        print(x.text, '!')

In [ ]:
spec_dict={}
spec_dict['Марка'] = driver.find_element_by_class_name('search-form-v2-mmm__breadcrumbs-item_type_mark').text
spec_dict['Модель'] = driver.find_element_by_class_name('search-form-v2-mmm__breadcrumbs-item_type_model').text
spec_dict['Поколение'] = driver.find_element_by_class_name('search-form-v2-mmm__breadcrumbs-item_type_generation').text
spec_dict['Тип кузова'] = driver.find_element_by_class_name('search-form-v2-mmm__breadcrumbs-item_type_configuration').text
spec_dict['Цена'] = driver.find_elements_by_xpath("//div[@class='catalog-generation__sales-info-block-link']/a[@data-bem=\""+str(a)+"\"]")

In [ ]:
spec_dict

In [ ]:
a ={'link':{'metrika':['listing','new_minmax_price']}}

In [ ]:
try:
    driver.find_element_by_xpath("//div[@class='catalog-generation__sales-info-block'][div[@class='catalog-generation__sales-info-block-title']='Новые']/div[@class='catalog-generation__sales-info-block-link']/a").text
except:
    print('-')

In [ ]:
a = driver.find_element_by_xpath("//div[@class='catalog-generation__sales-info-block'][div[@class='catalog-generation__sales-info-block-title']='С пробегом']/div[@class='catalog-generation__sales-info-block-link']/a").text


In [ ]:
(int(a.split('–')[0].replace(' ','')) + int(a.split('–')[1].replace(' ','').replace('₽','')))/2

In [ ]:
driver.get(all_models_links[145])

In [ ]:
data_spec = pd.DataFrame()



driver.get(all_models_links[145])
wait = WebDriverWait(driver, 10)


brand = driver.find_element_by_class_name('search-form-v2-mmm__breadcrumbs-item_type_mark').text
model = driver.find_element_by_class_name('search-form-v2-mmm__breadcrumbs-item_type_model').text
generation = driver.find_element_by_class_name('search-form-v2-mmm__breadcrumbs-item_type_generation').text
config = driver.find_element_by_class_name('search-form-v2-mmm__breadcrumbs-item_type_configuration').text

try:
    used_price_string = driver.find_element_by_xpath("//div[@class='catalog-generation__sales-info-block'][div[@class='catalog-generation__sales-info-block-title']='С пробегом']/div[@class='catalog-generation__sales-info-block-link']/a").text
    price = (int(a.split('–')[0].replace(' ','')) + int(a.split('–')[1].replace(' ','').replace('₽','')))/2
except:
    price = None

button = wait.until(ec.element_to_be_clickable((By.LINK_TEXT, 'Характеристики')))
ActionChains(driver).click(button).perform()


time.sleep(5)


models_repres = driver.find_elements_by_css_selector(".catalog-table__inner .link_theme_auto")
print(len(models_repres))
for i in range(len(models_repres)):
    driver.find_elements_by_css_selector(".catalog-table__inner .link_theme_auto")[i].click()
    #считываем характеристики
    spec_dict={}
    spec_dict['Марка'] = brand
    spec_dict['Модель'] = model
    spec_dict['Поколение'] = generation
    spec_dict['Тип кузова'] = config
    spec_dict['Цена'] = price
    
    time.sleep(5)
    
    for group in driver.find_elements_by_class_name('catalog__details-group'):
        a = group.find_elements_by_class_name('list-values__label')
        b = group.find_elements_by_class_name('list-values__value')
    for x, y in zip(a,b):
        spec_dict[x.text] = [y.text]
        
        
    comp = driver.find_elements_by_link_text('Комплектации')
    try:
        if len(comp) != 0:
            time.sleep(5)
            comp[0].click()
            time.sleep(5)
            # cчитываем комплектацию
            spec_dict['Модель'] = driver.find_element_by_class_name('catalog__package-name').text
            spec_dict['Цена'] = int(driver.find_element_by_class_name('catalog__package-price').text.replace(' ','').replace('₽',''))
    except:
        pass
    
    button = wait.until(ec.element_to_be_clickable((By.LINK_TEXT, 'Характеристики')))
    ActionChains(driver).click(button).perform()
    
        
    temp_data = pd.DataFrame.from_dict(spec_dict)
    data_spec = data_spec.append(temp_data, sort=False)
data_spec = data_spec.reset_index()

In [6]:
data_spec = pd.read_csv('out.csv')
for link in all_models_links[1331:]:
    driver.get(link)
    time.sleep(random.randint(3, 7))
    
    
    wait = WebDriverWait(driver, 60)


    brand = driver.find_element_by_class_name('search-form-v2-mmm__breadcrumbs-item_type_mark').text
    model = driver.find_element_by_class_name('search-form-v2-mmm__breadcrumbs-item_type_model').text
    generation = driver.find_element_by_class_name('search-form-v2-mmm__breadcrumbs-item_type_generation').text
    config = driver.find_element_by_class_name('search-form-v2-mmm__breadcrumbs-item_type_configuration').text

    try:
        used_price_string = driver.find_element_by_xpath("//div[@class='catalog-generation__sales-info-block'][div[@class='catalog-generation__sales-info-block-title']='С пробегом']/div[@class='catalog-generation__sales-info-block-link']/a").text
        price = (int(a.split('–')[0].replace(' ','')) + int(a.split('–')[1].replace(' ','').replace('₽','')))/2
    except:
        price = None

    button = wait.until(ec.element_to_be_clickable((By.LINK_TEXT, 'Характеристики')))
    ActionChains(driver).click(button).perform()

    wait.until(ec.presence_of_element_located((By.XPATH, "//dt[text()='Тип двигателя']")))
    time.sleep(random.randint(5, 7))


    models_repres = driver.find_elements_by_css_selector(".catalog-table__inner .link_theme_auto")
    
    for i in range(len(models_repres)):
        wait.until(ec.presence_of_element_located((By.XPATH, "//dt[text()='Тип двигателя']")))
        time.sleep(random.randint(5, 7))
        driver.find_elements_by_css_selector(".catalog-table__inner .link_theme_auto")[i].click()
        #считываем характеристики
        spec_dict={}
        spec_dict['Марка'] = brand
        spec_dict['Модель'] = model
        spec_dict['Поколение'] = generation
        spec_dict['Тип кузова'] = config
        spec_dict['Цена'] = price
        
        
        wait.until(ec.presence_of_element_located((By.XPATH, "//dt[text()='Тип двигателя']")))
        time.sleep(random.randint(5, 7))
        for group in driver.find_elements_by_class_name('catalog__details-group'):
            a = group.find_elements_by_class_name('list-values__label')
            b = group.find_elements_by_class_name('list-values__value')
            for x, y in zip(a,b):
                spec_dict[x.text] = [y.text]


        comp = driver.find_elements_by_link_text('Комплектации')
        try:
            if len(comp) != 0:
                time.sleep(5)
                comp[0].click()
                time.sleep(5)
                # cчитываем комплектацию
                spec_dict['Цена'] = int(driver.find_element_by_class_name('catalog__package-price').text.replace(' ','').replace('₽',''))
        except:
            pass

        button = wait.until(ec.element_to_be_clickable((By.LINK_TEXT, 'Характеристики')))
        ActionChains(driver).click(button).perform()


        temp_data = pd.DataFrame.from_dict(spec_dict)
        data_spec = data_spec.append(temp_data, sort=False)
        clear_output(wait=True)
        display(data_spec)
        data_spec.to_csv('out.csv', index=False)
        print(link)
data_spec = data_spec.reset_index()

,Марка,Модель,Поколение,Тип кузова,Цена,Страна марки,Класс автомобиля,Количество дверей,Количество мест,Длина,Ширина,Высота,Колёсная база,Ширина передней колеи,Ширина задней колеи,Размер колёс,"Объем багажника мин/макс, л","Объём топливного бака, л","Снаряженная масса, кг","Полная масса, кг",Коробка передач,Количество передач,Тип привода,Тип передней подвески,Тип задней подвески,Передние тормоза,Задние тормоза,"Максимальная скорость, км/ч","Разгон до 100 км/ч, с","Расход топлива, л город/трасса/смешанный",Марка топлива,Экологический класс,"Выбросы CO2, г/км",Тип двигателя,Расположение двигателя,"Объем двигателя, см³",Тип наддува,"Максимальная мощность, л.с./кВт при об/мин","Максимальный крутящий момент, Н*м при об/мин",Расположение цилиндров,Количество цилиндров,Число клапанов на цилиндр,Степень сжатия,"Диаметр цилиндра и ход поршня, мм",Клиренс,Система питания двигателя,"Расход топлива, л смешанный",Unnamed: 47,Оценка безопасности,Название рейтинга,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55,"Расход топлива, л трасса/смешанный","Расход топлива, л город/трасса","Расход топлива, л трасса",Unnamed: 59,Unnamed: 60,"Расход топлива, л город",Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65,Unnamed: 66,Unnamed: 67,Unnamed: 68,Unnamed: 69,Unnamed: 70,Unnamed: 71,Unnamed: 72,Unnamed: 73,Unnamed: 74,Unnamed: 75,Unnamed: 76,Unnamed: 77,Unnamed: 78,"Расход топлива, л город/смешанный",Unnamed: 80,Unnamed: 81,Unnamed: 82,Unnamed: 83,
0,Alpina,B5,G30/G31 2017 – н.в.,Седан,NaN,Германия,E,4,5,4956,1868,1466,2975,1611,1596,255/35/R20 295/30/R20,530,68,2045,2480,автомат,8,полный,"независимая, пружинная","независимая, пружинная",дисковые вентилируемые,дисковые вентилируемые,330,3.5,15.4/8.3/10.9,АИ-98,Euro 6,248,бензин,"переднее, продольное",4395,турбонаддув,608 / 447 при 5500 – 6500,800 при 2000 – 5000,V-образное,8,4,10.5,89 × 88.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Alpina,B5,G30/G31 2017 – н.в.,Универсал 5 дв.,NaN,Германия,E,5,5,4956,1868,1466,2975,1611,1596,255/35/R20 285/30/R20,570/1700,68,2150,2620,автомат,8,полный,"независимая, пружинная","независимая, пружинная",дисковые вентилируемые,дисковые вентилируемые,322,3.7,15.4/8.3/10.9,АИ-98,Euro 6,248,бензин,"переднее, продольное",4395,турбонаддув,608 / 447 при 5500 – 6500,800 при 2000 – 5000,V-образное,8,4,10.5,89 × 88.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Alpina,B7,G11 2016 – 2019,Седан,NaN,Германия,F,4,5,5250,1902,1491,3210,1622,1619,255/40/R20 295/35/R20,515,78,2110,2735,автомат,8,полный,"независимая, пневмоэлемент","независимая, пневмоэлемент",дисковые вентилируемые,дисковые вентилируемые,310,3.7,14.7/7.9/10.4,АИ-98,Euro 6,242,бензин,"переднее, продольное",4395,турбонаддув,608 / 447 при 5750 – 6250,800 при 3000 – 5000,V-образное,8,4,10,89 × 88.3,110,непосредственный впрыск (прямой),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Alpina,D3,F30 Рестайлинг 2017 – 2019,Универсал 5 дв.,NaN,Германия,D,5,5,4632,1811,1428,2810,1539,1556,245/35/R19 265/35/R19,495/1500,57,1730,2205,автомат,8,задний,"независимая, пружинная","независимая, пружинная",дисковые вентилируемые,дисковые вентилируемые,274,4.6,6.1/5/5.4,ДТ,Euro 6,142,дизель,"переднее, продольное",2993,турбонаддув,350 / 257 при 4000,700 при 1500 – 3000,рядное,6,4,16.5,84 × 90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Alpina,D3,F30 Рестайлинг 2017 – 2019,Универсал 5 дв.,NaN,Германия,D,5,5,4632,1811,1428,2810,1539,1556,245/30/R20 265/30/R20,495/1500,57,1800,2275,автомат,8,полный,"независимая, пружинная","независимая, пружинная",диско

https://auto.ru/catalog/cars/uaz/pickup/20875116/20875151/?year_from=2019


In [5]:
all_models_links.index('https://auto.ru/catalog/cars/volkswagen/caddy/20559423/20559489/?year_from=2019')

1331

In [ ]:
data_spec = pd.read_csv('out.csv')

In [ ]:
data_spec

In [ ]:
len(all_models_links)